In [1]:
import pandas as pd
import os

## ETL das bases de dados

In [3]:
clientes = pd.read_csv("../BASES/Clientes.csv", sep=";")
estoque = pd.read_csv("../BASES/Estoque.csv", sep=";")
fornecedores = pd.read_csv("../BASES/Fornecedores.csv", sep=";", encoding="ISO-8859-1")
produtos = pd.read_csv("../BASES/Produtos.csv", sep=";", encoding="ISO-8859-1")
vendas = pd.read_csv("../BASES/Vendas.csv", sep=";")

In [5]:
#Criação da pasta "TRATADOS" para armazenamento das tabelas tratadas:
os.makedirs("../TRATADOS", exist_ok=True)

### ----------

#### 1.Clientes

In [6]:
clientes.head(2)

,id,name,register_number,phone,email,address,city,zip_code
0,1,Cliente 1,05533520586545,(40) 3401-5150,cliente1@example.com,"Rua Tiradentes, 926",Pinhais,54289891
1,2,Cliente 2,61776869155,(56) 4159-1288,cliente2@example.com,"Rua Tiradentes, 645",Londrina,38512465


In [7]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               137 non-null    int64 
 1   name             137 non-null    object
 2   register_number  137 non-null    object
 3   phone            137 non-null    object
 4   email            137 non-null    object
 5   address          137 non-null    object
 6   city             137 non-null    object
 7   zip_code         137 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 8.7+ KB


In [8]:
#Verificar se há duplicatas:
print("id:",clientes["id"].duplicated().sum())
print("name:",clientes["name"].duplicated().sum())
print("register_number:",clientes["register_number"].duplicated().sum())

id: 0
name: 0
register_number: 0


In [9]:
#Converção da coluna "register_number" para str, evitando valores em not. cientifica:
clientes["register_number"] = clientes["register_number"].astype(str).apply(
    lambda x: "{:.0f}".format(float(x.replace(",", "."))) if "E" in x else x
)

In [10]:
# Adicionar dados de estad0, país e localidade geral:
clientes["state"] = "PR"
clientes["country"] = "Brasil"
clientes["local"] = clientes["city"] + " - " + clientes["state"] + " - " + clientes["country"]

In [11]:
# Criar uma coluna de endereço completo:
clientes['complete_address'] = clientes['address'] + ', ' + clientes['local']

In [13]:
clientes.to_csv("../TRATADOS/Clientes_tratado.csv", sep=";", index=False)

### ----------

#### 2.Estoque

In [14]:
estoque.head(2)

,product_id,stock_qty,updated_at,supplier_id
0,1,53,01/10/2024,1
1,2,89,17/04/2023,21


In [15]:
estoque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   50 non-null     int64 
 1   stock_qty    50 non-null     int64 
 2   updated_at   50 non-null     object
 3   supplier_id  50 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ KB


In [16]:
#Verificar se há duplicatas:
print("product_id:", estoque["product_id"].duplicated().sum())
print("supplier_id:", estoque["supplier_id"].duplicated().sum())

product_id: 0
supplier_id: 38


In [17]:
#Converter data para formato "AAA-MM-DD":
estoque["updated_at"] = pd.to_datetime(estoque["updated_at"], format="%d/%m/%Y").dt.date
estoque["updated_at"].head(3)

0    2024-10-01
1    2023-04-17
2    2020-06-13
Name: updated_at, dtype: object

In [18]:
estoque.to_csv("../TRATADOS/Estoque_tratado.csv", sep=";", index=False)

### ----------

#### 3.Fornecedores

In [19]:
fornecedores.head(2)

,supplier_id,contractor_name,register_number,address,city,zip_code,buyed_qty
0,1,Nogueira S.A.,02.589.476/0001-35,"Rua XV de Novembro, 123",Curitiba - PR,80010-000,72
1,7,Ramos Vieira e Filhos,21.395.478/0001-38,"Avenida Paraná, 456",Londrina - PR,86010-000,447


In [20]:
fornecedores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   supplier_id      12 non-null     int64 
 1   contractor_name  12 non-null     object
 2   register_number  12 non-null     object
 3   address          12 non-null     object
 4   city             12 non-null     object
 5   zip_code         12 non-null     object
 6   buyed_qty        12 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 804.0+ bytes


In [21]:
#Verificar se há duplicatas:
print("supplier_id:", fornecedores["supplier_id"].duplicated().sum())
print("contractor_name:", fornecedores["contractor_name"].duplicated().sum())

supplier_id: 0
contractor_name: 0


In [22]:
#Criação da coluna da unidade federativa do estado a partir da coluna "city":
fornecedores["uf"] = fornecedores["city"].str.split(" - ").str[-1]

In [23]:
fornecedores.to_csv("../TRATADOS/Fornecedores_tratado.csv", sep=";", index=False)

### ----------

#### 4.Produtos

In [24]:
produtos.head(2)

,product_id,product_name,category,value,supplier_id
0,1,Dolore SSD,Acessorios,4382.60,1
1,2,Ex HD,Acessorios,4145.27,5


In [25]:
produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    50 non-null     int64  
 1   product_name  50 non-null     object 
 2   category      50 non-null     object 
 3   value         50 non-null     float64
 4   supplier_id   50 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.1+ KB


In [26]:
#Verificar se há duplicatas:
print("product_id:", produtos["product_id"].duplicated().sum())
print("product_name:", produtos["product_name"].duplicated().sum())
print("supplier_id:", produtos["supplier_id"].duplicated().sum())

product_id: 0
product_name: 0
supplier_id: 38


In [27]:
produtos.to_csv("../TRATADOS/Produtos_tratado.csv", sep=";", index=False)

### ----------

#### 5.Vendas

In [28]:
vendas.head(2)

,id,client_id,product_id,quantity,sale_date,total_value
0,1,116,38,2,10/01/2025,5325.38
1,2,39,30,1,09/08/2024,1536.48


In [29]:
vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           100 non-null    int64  
 1   client_id    100 non-null    int64  
 2   product_id   100 non-null    int64  
 3   quantity     100 non-null    int64  
 4   sale_date    100 non-null    object 
 5   total_value  100 non-null    float64
dtypes: float64(1), int64(4), object(1)
memory usage: 4.8+ KB


In [30]:
#Verificar se há duplicatas:
print("id:", vendas["id"].duplicated().sum())
print("client_id:", vendas["client_id"].duplicated().sum())
print("product_id:", vendas["product_id"].duplicated().sum())

id: 0
client_id: 25
product_id: 57


In [31]:
#Converter data para formato "AAA-MM-DD":
vendas["sale_date"] = pd.to_datetime(vendas["sale_date"], format="%d/%m/%Y")
vendas["sale_date"].head(3)

0   2025-01-10
1   2024-08-09
2   2024-12-04
Name: sale_date, dtype: datetime64[ns]

In [32]:
vendas.to_csv("../TRATADOS/Vendas_tratado.csv", sep=";", index=False)